In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")

cap = cv2.VideoCapture("dataset/1.Training/video/fist/C032_A24_SY21_P01_S01_01DAS.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for kp in results[0].keypoints:
        coords = kp.xy.cpu().numpy()
        print("Keypoints:", coords)

    annotated_frame = results[0].plot()

    cv2.imshow("YOLOv8 Pose Estimation", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 3 persons, 98.2ms
Speed: 4.0ms preprocess, 98.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Keypoints: [[[     971.13      311.65]
  [     977.98      298.94]
  [     957.15      300.66]
  [          0           0]
  [     922.98      302.29]
  [     999.31      368.12]
  [     889.52      373.84]
  [     1022.8      459.01]
  [     889.13      473.96]
  [     1015.9      540.99]
  [     941.77      552.34]
  [      988.1      555.96]
  [     913.89      560.01]
  [     998.25      691.75]
  [     919.95      690.71]
  [     1009.2      819.01]
  [     932.11      809.86]]]
Keypoints: [[[     742.41      336.16]
  [      750.6      323.44]
  [      726.1      325.63]
  [          0           0]
  [     690.02      334.96]
  [     769.05      412.58]
  [     664.42      421.41]
  [     796.19      516.37]
  [     650.95      535.57]
  [     805.35      603.89]
  [      667.4      638.11]
  [     763.79      607.35]
  [     696.64      613.76]
  [     7

In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")

cap = cv2.VideoCapture("dataset/1.Training/video/bat/C031_A23_SY21_P01_S06_10DAS.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)


    for kp in results[0].keypoints:
        coords = kp.xy.cpu().numpy()
        print("Keypoints:", coords)

    annotated_frame = results[0].plot()

    cv2.imshow("YOLOv8 Pose Estimation", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 2 persons, 98.3ms
Speed: 10.4ms preprocess, 98.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Keypoints: [[[     696.86      514.13]
  [     693.05      503.19]
  [     678.93      513.43]
  [          0           0]
  [     651.28      540.88]
  [     708.02      549.79]
  [     667.46      622.44]
  [     721.88      509.84]
  [     726.11      713.43]
  [     699.92      459.87]
  [     798.67      762.68]
  [     798.08       691.4]
  [     766.98      726.59]
  [     855.11      774.72]
  [     841.11      808.19]
  [     871.35      866.87]
  [     855.58      894.29]]]
Keypoints: [[[     1164.9      372.72]
  [     1173.4      364.46]
  [          0           0]
  [     1208.7      360.86]
  [          0           0]
  [     1228.8      410.29]
  [     1226.8      386.99]
  [     1248.1      488.58]
  [          0           0]
  [     1249.9      567.66]
  [       1235      538.46]
  [     1254.6      523.81]
  [       1254      515.57]
  [     

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")

video_label_list = [
    ("dataa/normal.mp4", 0),
    ("dataa/not_normal.mp4", 1)
]

X = []
y = []

for video_path, label in video_label_list:
    cap = cv2.VideoCapture(video_path)
    print(f"{video_path}")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)

        for kp in results[0].keypoints:
            coords = kp.xy.cpu().numpy().flatten()
            if len(coords) == 34:
                X.append(coords)
                y.append(label)

        break

    cap.release()

X = np.array(X)
y = np.array(y)

np.save("X_keypoints.npy", X)
np.save("y_labels.npy", y)

print(f"X.shape = {X.shape}, y.shape = {y.shape}")


🎥 처리 중: dataa/normal.mp4

0: 384x640 2 persons, 52.6ms
Speed: 9.0ms preprocess, 52.6ms inference, 4.9ms postprocess per image at shape (1, 3, 384, 640)
🎥 처리 중: dataa/not_normal.mp4

0: 384x640 4 persons, 62.6ms
Speed: 7.3ms preprocess, 62.6ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)
✅ 저장 완료: X.shape = (6, 34), y.shape = (6,)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
import pickle

X = np.load("X_keypoints.npy")
y = np.load("y_labels.npy")

print(f"📊 데이터셋: X={X.shape}, y={y.shape}, 클래스 분포={np.unique(y, return_counts=True)}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu',
                      solver='adam', max_iter=300, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("분류 결과:")
print(classification_report(y_test, y_pred))
print(f"정확도: {accuracy_score(y_test, y_pred):.4f}")

with open("violence_classifier.pkl", "wb") as f:
    pickle.dump(model, f)

📊 데이터셋: X=(6, 34), y=(6,), 클래스 분포=(array([0, 1]), array([2, 4]))
🔍 분류 결과:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

✅ 정확도: 0.5000
💾 모델 저장 완료: violence_classifier.pkl


c:\Users\dlals\anaconda3\envs\behav\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dlals\anaconda3\envs\behav\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dlals\anaconda3\envs\behav\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
